In [60]:
# Copied from https://www.kaggle.com/alhalimi/tokenization-and-word-embedding-compatibility
#find words in common between a given embedding and our vocabulary
def compare_vocab_and_embeddings(vocab, embeddings_index):
    """
    :params vocab: our corpus vocabulary (a dictionary of word frquencies)
            embeddings_index: a genim object containing loaded embeddings.
    :returns in_common: words in common,
             in_common_freq: total frequency in the corpus vocabulary of 
                             all words in common
             oov: out of vocabulary words
             oov_frequency: total frequency in vocab of oov words
    """
    in_common={}
    oov=[]
    in_common=[]
    in_common_freq = 0
    oov_freq = 0
    
    # Compose the vocabulary given the sentence tokens
    #vocab = get_vocab(sentences)

    for word in vocab:
        if word in embeddings_index:
            in_common.append(word)
            in_common_freq += vocab[word]
        else: 
            oov.append(word)
            oov_freq += vocab[word]
    
    print('Found embeddings for {:.2%} of vocab'.format(len(in_common) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(in_common_freq / (in_common_freq + oov_freq)))

    return sorted(in_common)[::-1], sorted(oov)[::-1], in_common_freq, oov_freq, vocab

# print the list of out-of-vocabulary words sorted by their frequency in teh training text
def show_oov_words(oov, vocab,  num_to_show=15):
    # Sort oov words by their frequency in the text
    sorted_oov= sorted(oov, key =lambda x: vocab[x], reverse=True )

    # Show oov words and their frequencies
    if (len(sorted_oov)>0):
        print("oov words:")
        for word in sorted_oov[:num_to_show]:
            print("%s\t%s"%(word, vocab[word]))
    else:
        print("No words were out of vocabulary.")
        
    return len(sorted_oov);

In [65]:
!rm -rf lang2sign
!git clone https://github.com/monkeyhippies/lang2sign.git
!cd lang2sign && git checkout open-nmt

Cloning into 'lang2sign'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 339 (delta 49), reused 63 (delta 28), pack-reused 252
Receiving objects: 100% (339/339), 319.30 KiB | 0 bytes/s, done.
Resolving deltas: 100% (171/171), done.
Branch open-nmt set up to track remote branch open-nmt from origin.
Switched to a new branch 'open-nmt'


In [66]:
!cd lang2sign && make data pretrained-embeddings preprocess

./utils/download_en_asl_gloss_data.sh
wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)
--2019-09-22 16:44:28--  https://raw.githubusercontent.com/monkeyhippies/speech2signs-2017-nmt/master/ASLG-PC12/train.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3765908 (3.6M) [application/octet-stream]
Saving to: ‘data/raw/lang2gloss/enasl/train.tgz’

train.tgz           100%[===================>]   3.59M  --.-KB/s    in 0.1s    

2019-09-22 16:44:28 (34.8 MB/s) - ‘data/raw/lang2gloss/enasl/train.tgz’ saved [3765908/3765908]

train/
train/ENG-ASL_Train_0.046.asl
train/ENG-ASL_Train_0.046.en
wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)
--2019-09-22 16:44:29--  https://raw.githubusercontent.

In [67]:
case_sensitive = False

replacements = {
    " refore ": " therefore ",
    " toger ": " together ",
    " strengn ": "strengthen ",
    " russium ": " russia "
}
files = [
    'lang2sign/data/preprocessed/lang2gloss/enasl/train/ENG-ASL_Train_0.046.asl',
    'lang2sign/data/preprocessed/lang2gloss/enasl/train/ENG-ASL_Train_0.046.en',
]

embedding_file = 'lang2sign/pretrained-embeddings/glove.6B.300d.txt.trimmed.vocab'
with open(embedding_file, "r") as file_obj:
    embeddings = set(file_obj.read().split())

vocab = {}
corpus = ""
for file in files:
    with open(file, "r") as file_obj:
        corpus += file_obj.read()

if not case_sensitive:
    corpus = corpus.lower()

for key, value in replacements.items():
    corpus = corpus.replace(key, value)

words = corpus.split()


for word in words:
    if word in vocab:
        vocab[word] += 1
    else:
        vocab[word] = 1

In [68]:
in_common, oov, in_common_freq, oov_freq, vocab = compare_vocab_and_embeddings(vocab, embeddings)

Found embeddings for 84.65% of vocab
Found embeddings for  99.54% of all text


In [69]:
show_oov_words(oov, vocab)

oov words:
mselves	176
neverless	154
furrmore	142
codecision	142
romanium	141
orwise	125
200written	104
croatium	103
solvit	94
neir	88
tostrengthen	83
georgium	78
haitus	77
bosnium	62
taxpayersmoney	59


3644

In [ ]:
!cat lang2sign/data/raw/lang2gloss/enasl/train/ENG-ASL_Train_0.046.asl | grep -in russia

In [57]:
"russia" in embeddings

True